In [20]:
from bs4 import BeautifulSoup
import base64
import wget
import requests
import unicodecsv as csv
import datetime

In [21]:
total_articles = [] # it will save all publications

In [22]:
## get days
iterator_day = datetime.date(2019, 7, 29)
last_publication = datetime.date(2019, 6, 29)

In [23]:
array_days = [str(iterator_day)]
while(iterator_day >= last_publication):
    iterator_day = iterator_day - datetime.timedelta(days=1)
    array_days.append(str(iterator_day))

In [24]:
def save_csv(articles):
    with open('el_comercio_7_29.csv', 'wb') as f:
            dict_writer = csv.DictWriter(
                f,
                [
                    'Titulo',
                    'Resumen',
                    'Categoria', 
                    'Autor', 
                    'Fecha', 
                    'Contenido', 
                    'Etiquetas',
                    'Enlace'
                ],
                encoding='utf-8')
            dict_writer.writeheader()
            dict_writer.writerows(articles)

In [25]:
def get_title(content_soup):
    if content_soup.find('h1', {'class': 'story-header__news-title'}) is not None:
        return content_soup.find('h1', {'class': 'story-header__news-title'}).text
    
    return;

In [26]:
def get_summary(content_soup):
    if content_soup.find('h2', {'class': 'story-header__news-summary'}) is not None:
        return content_soup.find('h2', {'class': 'story-header__news-summary'}).text
    
    return;

In [27]:
def get_category(content_soup):
    if content_soup.find('div', {'class': 'story-header__title-section'}) is not None:
        return content_soup.find('div', {'class': 'story-header__title-section'}).text
            
    return;

In [28]:
def get_author(content_soup):
    if content_soup.find('a', {'class': 'story-content__author-link'}) is not None:
        return content_soup.find('a', {'class': 'story-content__author-link'}).text
    
    return;

In [29]:
def get_date(content_soup):
    if content_soup.find('div', {'class': 'story-content__date'}) is not None:
        if content_soup.find('div', {'class': 'story-content__date'}).find('time') is not None:
            return content_soup.find('div', {'class': 'story-content__date'}).find('time')['datetime']
    
    return;

In [30]:
def get_content(content_soup):
    if content_soup.find('div', {'class': 'story-content__content'}) is not None:
        if len(content_soup.find('div', {'class': 'story-content__content'}).findAll('p', {'class': 'story-content__font--secondary'})) > 0:
            return content_soup.find('div', {'class': 'story-content__content'}).findAll('p', {'class': 'story-content__font--secondary'})
    
    return [];

In [31]:
def get_tags(content_soup):
    if len(content_soup.findAll('a', {'class': 'story-tags__link'})) > 0:
        return content_soup.findAll('a', {'class': 'story-tags__link'})
    
    return [];

In [32]:
def interate_articles_and_save(articles):
    cont = 0
    for article in articles:
        print('-------------------- Article ----------------------')
        url_article = 'https://elcomercio.pe' + article.find('a', {'class': 'story-item__title'})['href']
        content_article = requests.get(url_article).content
        soup_article = BeautifulSoup(content_article,'lxml')
        print("URL Article===>" + str(url_article))
        
        title = get_title(soup_article)
        summary = get_summary(soup_article)
        category = get_category(soup_article)
        author = get_author(soup_article)
        date = get_date(soup_article)
        text_content = get_content(soup_article)
        content_article = []
        for text in text_content:    
            content_article.append(text.text)

        tags_article = get_tags(soup_article)
        tags = []
        for tag in tags_article:
            tags.append(tag.text)

        total_articles.extend([
            {
                'Titulo':str(title), 
                'Resumen': str(summary), 
                'Categoria':str(category), 
                'Autor':str(author),
                'Fecha':str(date),
                'Contenido':str(content_article),
                'Etiquetas':str(tags),
                'Enlace':url_article
            }
        ])
        save_csv(total_articles)
        cont = cont + 1
        print('-----------------------------------  ' + str(cont*100/len(articles)) + "%")
        print('-------------------- End Article ----------------------')

In [33]:
def iterate_publications(days):
    for day in days:
        print('-------------------- Day ----------------------')
        url_day = 'https://elcomercio.pe/archivo/todas/' + day
        content_day = requests.get(url_day).content
        soup_day = BeautifulSoup(content_day,'lxml')
        print("URL Day===>" + str(url_day))
        
        articles_day = soup_day.findAll('div', {'class' : 'story-item'})
        interate_articles_and_save(articles_day)
        #try:
            #interate_articles_and_save(articles_day)
        #except:
            #print('Error para obtener pagina====> ' + url_day)
        print('-------------------- End Day ----------------------')

In [34]:
iterate_publications(array_days)

-------------------- Day ----------------------
URL Day===>https://elcomercio.pe/archivo/todas/2019-07-29
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/mundo/mexico/clima-mexico-pronostico-martes-30-julio-2019-noticia-nndc-660177-noticia/
-----------------------------------  1.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/videos/global/papa-francisco-prostitucion-vicio-repugnante-video-nnav-noticia-660175-noticia/
-----------------------------------  2.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/gastronomia/peruana/astrid-gaston-25-anos-noticia-ecpm-659675-noticia/
-----------------------------------  3.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcome

URL Article===>https://elcomercio.pe/panamericanos-lima-2019/honduras-vs-jamaica-vivo-online-grupo-b-futbol-masculino-juegos-panamericanos-lima-2019-noticia-liver-sports-event-nczd-659911-noticia/
-----------------------------------  28.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/mundo/actualidad/huracan-maria-puerto-rico-miles-botellas-agua-donada-damnificados-aparecen-finca-noticia-660146-noticia/
-----------------------------------  29.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/lima/sucesos/senamhi-temperatura-minima-lima-descendera-13-c-hoy-lunes-29-julio-noticia-nndc-659893-noticia/
-----------------------------------  30.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/videos/e

URL Article===>https://elcomercio.pe/viu/moda/cinco-pantalones-temporada-super-comodos-moda-noticia-nndc-656861-noticia/
-----------------------------------  54.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/videos/global/espana-pasean-muerte-honor-santa-marta-video-nnav-noticia-660114-noticia/
-----------------------------------  55.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/peru/la-libertad/libertad-ultimas-24-horas-ocurrieron-dos-asesinatos-trujillo-noticia-660104-noticia/
-----------------------------------  56.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/viu/actitud-viu/camila-garay-bailarina-ballet-suena-representarnos-vancouver-video-noticia-660108-noticia/
------------------

URL Article===>https://elcomercio.pe/mundo/eeuu/santino-william-legan-tiroteo-california-identifica-autor-balacera-publico-fotos-instagram-ataque-noticia-nndc-660049-noticia/
-----------------------------------  81.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/videos/trailers/evangelion-3-0-1-0-teaser-trailer-oficial-2020-video-viral-hideaki-anno-studios-khara-gainax-shinji-ikari-asuka-langley-rey-ayanami-nnda-nnrt-noticia-660075-noticia/
-----------------------------------  82.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/videos/pais/opinion-gente-desfile-patrio-video-nnav-noticia-660071-noticia/
-----------------------------------  83.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/pan

URL Article===>https://elcomercio.pe/videos/deportes/historia-gladys-tejeda-diego-elias-final-dorado-video-nnav-noticia-659909-noticia/
-----------------------------------  9.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/mundo/mexico/clima-mexico-pronostico-lunes-29-julio-2019-noticia-659908-noticia/
-----------------------------------  10.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/deporte-total/futbol-mundial/peru-vs-argentina-vivo-online-ver-gratis-tv-endirecto-2019-via-latina-juegan-grupo-b-futbol-femenino-lima-2019-resumen-goles-video-ar-col-cl-pe-noticia-nczd-659676-noticia/
-----------------------------------  11.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/lima/linea-amarill

...

URL Article===>https://elcomercio.pe/peru/tumbes/gobernador-tumbes-pide-desatender-migracion-venezolana-noticia-ecpm-656077-noticia/
-----------------------------------  81.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/tvmas/famosos/faceapp-maria-pia-copello-beto-ortiz-otros-artistas-sumaron-nueva-tendencia-noticia-nndc-656073-noticia/
-----------------------------------  82.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/tecnologia/videojuegos/pes-da-golpe-juventus-llamara-piamonte-calcio-fifa-20-perder-licencia-noticia-655890-noticia/
-----------------------------------  83.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/deporte-total/amir-khan-confirmo-pelea-manny-pacquiao-noviembre-ara

URL Article===>https://elcomercio.pe/opinion/colaboradores/kuroiwa-vida-dedicada-combatir-desastre-fabiola-leon-velarde-noticia-ecpm-655414-noticia/
-----------------------------------  9.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/opinion/mirada-de-fondo/ruido-andres-calderon-noticia-ecpm-655417-noticia/
-----------------------------------  10.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/tvmas/series/seinfeld-actor-charles-levin-hallado-muerto-dias-busqueda-noticia-655714-noticia/
-----------------------------------  11.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/casa-y-mas/ideas-y-diseno/descubre-trasformacion-camioneta-hospedaje-fotos-noticia-655675-noticia/
-------------------

URL Article===>https://elcomercio.pe/mundo/eeuu/ee-uu-investigan-62-miembros-patrulla-fronteriza-burlas-migrantes-redes-noticia-655760-noticia/
-----------------------------------  38.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/ruedas-tuercas/automotriz/nuevo-audi-q3-llega-peru-e-inicia-preventa-digital-fotos-noticia-655519-noticia/
-----------------------------------  39.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/luces/musica/youtube-avril-lavigne-sorprende-videoclip-i-fell-in-love-with-the-devil-fotos-video-noticia-nndc-655755-noticia/
-----------------------------------  40.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/deporte-total/futbol-mundial/barcelona-inaugurara-estadio-j

URL Article===>https://elcomercio.pe/lima/judiciales/jorge-munoz-linea-amarilla-atiende-pedido-mml-reducir-tarifa-peajes-noticia-nndc-655725-noticia/
-----------------------------------  66.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/panamericanos-lima-2019/lima-2019-ceremonia-apertura-busca-deslumbrar-mundo-naturaleza-cultura-peru-noticia-nndc-655735-noticia/
-----------------------------------  67.0%
-------------------- End Article ----------------------
-------------------- Article ----------------------


ConnectionError: HTTPSConnectionPool(host='elcomercio.pe', port=443): Max retries exceeded with url: /videos/pais/policia-enfrenta-balazos-delincuentes-video-noticia-655734-noticia/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f1e29f7fac8>: Failed to establish a new connection: [Errno -2] Name or service not known'))